# Multiple Linear Regression: Racial Bias in Sentencing

**Purpose:** Test if ethnicity predicts sentence length after controlling for legally relevant factors

**Research Question:** Do Black/Hispanic defendants receive longer sentences than White defendants with similar offense profiles and suitability scores?

**CRJA Connection:** Tests if "similarly situated" defendants receive different treatment based on race

## Step 1: Import Libraries

## Configuration: Data Source

This notebook uses the output from `03_prepare_regression_data.ipynb`

**Options:**
- **GitHub**: Load from Redo.io's prepared dataset
- **Local**: Use your own prepared data from `outputs/regression_analysis_data.csv`

In [ ]:
# Data source configuration
USE_GITHUB = True  # Set to False to use local prepared data

if USE_GITHUB:
    print("Using GitHub prepared dataset")
    DATA_PATH = "https://raw.githubusercontent.com/redoio/resentencing_data_initiative/main/outputs/regression_analysis_data.csv"
else:
    print("Using local prepared dataset")
    from pathlib import Path
    outputs_dir = Path("../outputs")
    DATA_PATH = outputs_dir / "regression_analysis_data.csv"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats

In [ ]:
# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## Step 2: Load Data

In [ ]:
# Load prepared regression data
data = pd.read_csv(DATA_PATH)
print(f"Loaded {len(data):,} records for analysis")

In [ ]:
data.shape

In [ ]:
data.head()

## Step 3: Exploratory Data Analysis

In [ ]:
# Summary statistics for key variables
data[['aggregate sentence in months', 'score', 'desc_nonvio_curr', 'desc_nonvio_past']].describe()

In [ ]:
# Ethnicity distribution
data['ethnicity'].value_counts()

In [ ]:
# Offense table distribution
data['offense_table'].value_counts()

### Visualize: Sentence Distribution by Ethnicity

In [ ]:
# Box plot: Sentence by ethnicity
plt.figure(figsize=(10, 6))
sns.boxplot(data=data, x='ethnicity', y='aggregate sentence in months')
plt.title('Sentence Length by Ethnicity (Unadjusted)', fontsize=14, fontweight='bold')
plt.xlabel('Ethnicity')
plt.ylabel('Sentence (months)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Mean sentence by ethnicity
mean_sentence = data.groupby('ethnicity')['aggregate sentence in months'].agg(['mean', 'median', 'count'])
mean_sentence

### Visualize: Suitability Score by Ethnicity

In [ ]:
# Do different ethnic groups have different suitability scores?
plt.figure(figsize=(10, 6))
sns.boxplot(data=data, x='ethnicity', y='score')
plt.title('Suitability Score by Ethnicity', fontsize=14, fontweight='bold')
plt.xlabel('Ethnicity')
plt.ylabel('Suitability Score (0-3)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Mean suitability by ethnicity
data.groupby('ethnicity')['score'].mean()

## Step 4: Prepare Data for Regression

In [ ]:
# Filter to main ethnic groups (sufficient sample size)
main_groups = ['Black', 'Hispanic', 'White']
reg_data = data[data['ethnicity'].isin(main_groups)].copy()

In [ ]:
# Set White as reference category (for interpretation)
reg_data['ethnicity'] = pd.Categorical(
    reg_data['ethnicity'],
    categories=['White', 'Black', 'Hispanic'],
    ordered=False
)

In [ ]:
# Check sample sizes
reg_data['ethnicity'].value_counts()

## Step 5: Model 1 - Baseline (Ethnicity + Suitability Score)

In [ ]:
# Formula: sentence ~ ethnicity + suitability score
model1 = smf.ols(
    'Q("aggregate sentence in months") ~ C(ethnicity) + score',
    data=reg_data
).fit()

In [ ]:
# View results
print(model1.summary())

### Interpret Model 1 Coefficients

In [ ]:
# Extract key coefficients
coef_black = model1.params['C(ethnicity)[T.Black]']
pval_black = model1.pvalues['C(ethnicity)[T.Black]']

coef_hispanic = model1.params['C(ethnicity)[T.Hispanic]']
pval_hispanic = model1.pvalues['C(ethnicity)[T.Hispanic]']

coef_score = model1.params['score']

print("Model 1 Interpretation:")
print(f"\nBlack defendants: {coef_black:+.2f} months (p = {pval_black:.4f})")
print(f"Hispanic defendants: {coef_hispanic:+.2f} months (p = {pval_hispanic:.4f})")
print(f"Suitability score effect: {coef_score:.2f} months per unit")
print(f"\nR-squared: {model1.rsquared:.3f}")

## Step 6: Model 2 - Add Offense Severity

In [ ]:
# Formula: sentence ~ ethnicity + suitability + offense table
model2 = smf.ols(
    'Q("aggregate sentence in months") ~ C(ethnicity) + score + C(offense_table)',
    data=reg_data
).fit()

In [ ]:
print(model2.summary())

In [ ]:
# Extract coefficients
coef_black_m2 = model2.params['C(ethnicity)[T.Black]']
pval_black_m2 = model2.pvalues['C(ethnicity)[T.Black]']

print("Model 2 Interpretation:")
print(f"\nBlack defendants: {coef_black_m2:+.2f} months (p = {pval_black_m2:.4f})")
print(f"R-squared: {model2.rsquared:.3f}")
print(f"\nChange from Model 1: {coef_black_m2 - coef_black:+.2f} months")

## Step 7: Model 3 - Add County Controls

In [ ]:
# Check number of counties
n_counties = reg_data['controlling case sentencing county'].nunique()
print(f"Number of unique counties: {n_counties}")

In [ ]:
# Keep only counties with sufficient cases (avoid overfitting)
county_counts = reg_data['controlling case sentencing county'].value_counts()
major_counties = county_counts[county_counts >= 50].index

reg_data_county = reg_data[reg_data['controlling case sentencing county'].isin(major_counties)].copy()
print(f"Using {len(major_counties)} major counties with 50+ cases")

In [ ]:
# Formula: sentence ~ ethnicity + suitability + offense + county
model3 = smf.ols(
    'Q("aggregate sentence in months") ~ C(ethnicity) + score + C(offense_table) + C(Q("controlling case sentencing county"))',
    data=reg_data_county
).fit()

In [ ]:
print(model3.summary())

In [ ]:
# Extract coefficients
coef_black_m3 = model3.params['C(ethnicity)[T.Black]']
pval_black_m3 = model3.pvalues['C(ethnicity)[T.Black]']

print("Model 3 Interpretation:")
print(f"\nBlack defendants: {coef_black_m3:+.2f} months (p = {pval_black_m3:.4f})")
print(f"R-squared: {model3.rsquared:.3f}")

## Step 8: Model Comparison

In [ ]:
# Compare models side-by-side
comparison = pd.DataFrame({
    'Model 1 (Baseline)': [coef_black, pval_black, model1.rsquared, len(reg_data)],
    'Model 2 (+ Offense)': [coef_black_m2, pval_black_m2, model2.rsquared, len(reg_data)],
    'Model 3 (+ County)': [coef_black_m3, pval_black_m3, model3.rsquared, len(reg_data_county)]
}, index=['Black Coefficient (months)', 'P-value', 'R-squared', 'N'])

comparison

### Visualize: Coefficient Stability Across Models

In [ ]:
# Plot how Black coefficient changes across models
models = ['Model 1\n(Baseline)', 'Model 2\n(+ Offense)', 'Model 3\n(+ County)']
coefficients = [coef_black, coef_black_m2, coef_black_m3]

# Calculate confidence intervals (95%)
ci_black_m1 = model1.conf_int().loc['C(ethnicity)[T.Black]']
ci_black_m2 = model2.conf_int().loc['C(ethnicity)[T.Black]']
ci_black_m3 = model3.conf_int().loc['C(ethnicity)[T.Black]']

errors = [
    [coef_black - ci_black_m1[0], ci_black_m1[1] - coef_black],
    [coef_black_m2 - ci_black_m2[0], ci_black_m2[1] - coef_black_m2],
    [coef_black_m3 - ci_black_m3[0], ci_black_m3[1] - coef_black_m3]
]

plt.figure(figsize=(10, 6))
plt.errorbar(models, coefficients, yerr=np.array(errors).T, fmt='o-', capsize=10, markersize=10, linewidth=2)
plt.axhline(y=0, color='red', linestyle='--', linewidth=1, label='No Effect')
plt.ylabel('Additional Months for Black Defendants', fontsize=12)
plt.xlabel('Model Specification', fontsize=12)
plt.title('Black Sentencing Coefficient Across Models\n(with 95% Confidence Intervals)', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## Step 9: Model Diagnostics

### Check 1: Residual Plot (Linearity Assumption)

In [ ]:
# Plot residuals vs fitted values
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Residuals vs Fitted
axes[0].scatter(model2.fittedvalues, model2.resid, alpha=0.5)
axes[0].axhline(y=0, color='red', linestyle='--')
axes[0].set_xlabel('Fitted Values')
axes[0].set_ylabel('Residuals')
axes[0].set_title('Residuals vs Fitted Values')
axes[0].grid(alpha=0.3)

# Q-Q plot (normality)
sm.qqplot(model2.resid, line='s', ax=axes[1])
axes[1].set_title('Q-Q Plot')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### Check 2: Multicollinearity (VIF)

In [ ]:
# Calculate Variance Inflation Factor
# VIF > 10 indicates problematic multicollinearity

# Create dummy variables for categorical predictors
X = pd.get_dummies(reg_data[['ethnicity', 'score', 'offense_table']], drop_first=True)
X = sm.add_constant(X)

# Calculate VIF
vif_data = pd.DataFrame()
vif_data['Variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data[vif_data['Variable'] != 'const']

### Check 3: Outliers (Cook's Distance)

In [ ]:
# Calculate Cook's distance
influence = model2.get_influence()
cooks_d = influence.cooks_distance[0]

# Plot
plt.figure(figsize=(12, 5))
plt.stem(np.arange(len(cooks_d)), cooks_d, markerfmt=',')
plt.axhline(y=4/len(cooks_d), color='red', linestyle='--', label='Threshold (4/n)')
plt.xlabel('Observation Index')
plt.ylabel("Cook's Distance")
plt.title("Cook's Distance - Influential Observations")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Count influential points
n_influential = (cooks_d > 4/len(cooks_d)).sum()
print(f"Influential observations (Cook's D > 4/n): {n_influential} ({n_influential/len(cooks_d)*100:.1f}%)")

## Step 10: Interaction Analysis (Does Bias Vary by Offense Type?)

In [ ]:
# Model with interaction term
model_interaction = smf.ols(
    'Q("aggregate sentence in months") ~ C(ethnicity) * C(offense_table) + score',
    data=reg_data
).fit()

In [ ]:
print(model_interaction.summary())

### Visualize: Interaction Plot

In [ ]:
# Calculate mean sentence by ethnicity and offense table
interaction_data = reg_data.groupby(['offense_table', 'ethnicity'])['aggregate sentence in months'].mean().reset_index()

# Plot
plt.figure(figsize=(12, 6))
for ethnicity in ['White', 'Black', 'Hispanic']:
    subset = interaction_data[interaction_data['ethnicity'] == ethnicity]
    plt.plot(subset['offense_table'], subset['aggregate sentence in months'], 
             marker='o', label=ethnicity, linewidth=2, markersize=8)

plt.xlabel('Offense Table', fontsize=12)
plt.ylabel('Mean Sentence (months)', fontsize=12)
plt.title('Interaction: Ethnicity × Offense Type\n(Non-parallel lines indicate interaction)', fontsize=14, fontweight='bold')
plt.legend(title='Ethnicity', fontsize=11)
plt.xticks(rotation=45)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Step 11: Subgroup Analysis (First-Time Offenders Only)

In [ ]:
# Identify first-time offenders (desc_nonvio_past is NaN or they have no prior history)
# Using high desc_nonvio_curr as proxy for less serious current offense
first_time = reg_data[reg_data['desc_nonvio_past'].isna() | (reg_data['desc_nonvio_past'] == 1.0)].copy()

print(f"First-time offender sample: {len(first_time):,} ({len(first_time)/len(reg_data)*100:.1f}%)")

In [ ]:
# Run regression on first-time offenders only
if len(first_time) > 100:
    model_firsttime = smf.ols(
        'Q("aggregate sentence in months") ~ C(ethnicity) + desc_nonvio_curr + C(offense_table)',
        data=first_time
    ).fit()
    
    print(model_firsttime.summary())
    
    coef_black_ft = model_firsttime.params['C(ethnicity)[T.Black]']
    print(f"\nFirst-time offenders - Black coefficient: {coef_black_ft:+.2f} months")
else:
    print("Insufficient sample size for first-time offender analysis")

## Step 12: Save Results

In [ ]:
# Extract key results into summary table
results_summary = pd.DataFrame({
    'Model': ['Model 1: Baseline', 'Model 2: + Offense', 'Model 3: + County'],
    'Black Coefficient': [coef_black, coef_black_m2, coef_black_m3],
    'P-value': [pval_black, pval_black_m2, pval_black_m3],
    'Significant': [pval_black < 0.05, pval_black_m2 < 0.05, pval_black_m3 < 0.05],
    'R-squared': [model1.rsquared, model2.rsquared, model3.rsquared],
    'N': [len(reg_data), len(reg_data), len(reg_data_county)]
})

In [ ]:
results_summary

In [ ]:
# Save to CSV
results_summary.to_csv(outputs_dir / "mlr_results_summary.csv", index=False)
print("Results saved to: mlr_results_summary.csv")

## Step 13: Write Interpretation for CRJA Filing

In [ ]:
# Generate plain-language interpretation
print("="*70)
print("INTERPRETATION FOR CRJA FILING")
print("="*70)

print(f"\nSample Size: {len(reg_data):,} individuals")
print(f"Ethnic Groups: Black ({reg_data[reg_data['ethnicity']=='Black'].shape[0]:,}), "
      f"Hispanic ({reg_data[reg_data['ethnicity']=='Hispanic'].shape[0]:,}), "
      f"White ({reg_data[reg_data['ethnicity']=='White'].shape[0]:,})")

print("\nFINDINGS:")
print(f"\n1. Unadjusted Disparity:")
print(f"   Black defendants receive sentences averaging {mean_sentence.loc['Black', 'mean'] - mean_sentence.loc['White', 'mean']:.1f} months longer than White defendants")

print(f"\n2. After Controlling for Suitability (Model 1):")
if pval_black < 0.05:
    print(f"   Black defendants receive {coef_black:+.1f} additional months (p = {pval_black:.4f})")
    print(f"   This disparity is STATISTICALLY SIGNIFICANT")
else:
    print(f"   Black defendants receive {coef_black:+.1f} additional months (p = {pval_black:.4f})")
    print(f"   This disparity is NOT statistically significant")

print(f"\n3. After Controlling for Suitability AND Offense Severity (Model 2):")
if pval_black_m2 < 0.05:
    print(f"   Black defendants receive {coef_black_m2:+.1f} additional months (p = {pval_black_m2:.4f})")
    print(f"   This disparity PERSISTS after accounting for offense severity")
else:
    print(f"   Black defendants receive {coef_black_m2:+.1f} additional months (p = {pval_black_m2:.4f})")
    print(f"   This disparity is NOT statistically significant")

print("\nCONCLUSION:")
if pval_black_m2 < 0.05:
    print(f"Among similarly situated defendants (same suitability score and offense severity),")
    print(f"Black defendants receive significantly longer sentences ({coef_black_m2:+.1f} months).")
    print(f"This constitutes evidence of racial disparity under the CRJA.")
else:
    print(f"After controlling for legally relevant factors, no statistically significant")
    print(f"racial disparity in sentence length was detected.")

print("\n" + "="*70)

## Summary

**What we tested:**
- Whether ethnicity predicts sentence length after controlling for:
  - Suitability/risk profile (score)
  - Offense severity (table A/B/C/D/E/F)
  - Geographic jurisdiction (county)

**Key findings:**
- Model 1: Baseline disparity
- Model 2: Disparity after offense controls (main model)
- Model 3: Disparity after county controls (robustness check)

**Diagnostics:**
- Checked linearity assumptions
- Tested for multicollinearity
- Identified influential outliers

**Next steps:**
- Logistic regression (wobbler charging decisions)
- Propensity score matching (alternative method)
- County-specific analyses (identify outlier jurisdictions)